In [1]:
import json
from es_connection import EsManagement
import os
import time

import warnings
warnings.filterwarnings('ignore')


In [2]:
with open('elasticsearch-config/address_mapping.json', encoding='utf-8') as f:
    address_mapping = json.load(f)

index_name = "address_index"

es_connection = EsManagement()
es_connection.clear_index(index_name=index_name)
es_connection.create_index(index_name=index_name, mapping=address_mapping)

{'acknowledged': True}
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'address_index'}


In [13]:
address_mapping

{'settings': {'index': {'analysis': {'filter': {'EL_synonym': {'type': 'synonym',
      'synonyms': ['خ => خیابان',
       'ک => کوچه',
       'ك => کوچه',
       'ب => بلوار',
       'پ => پلاک',
       'م => میدان',
       'بل => بلوار',
       'بن => بن بست',
       'خر => تهرانپارس']}},
    'analyzer': {'EL_address_analyzer': {'filter': ['lowercase',
       'elision',
       'asciifolding',
       'EL_synonym'],
      'type': 'custom',
      'tokenizer': 'standard'}}}}},
 'mappings': {'properties': {'address': {'type': 'text',
    'analyzer': 'EL_address_analyzer'},
   'building_no': {'type': 'keyword'},
   'unit': {'type': 'text'}}}}

In [3]:
print(
  json.dumps(
    es_connection.es.indices.get_mapping(index=index_name), 
    indent=1)
)

{
 "address_index": {
  "mappings": {
   "properties": {
    "address": {
     "type": "text",
     "analyzer": "EL_address_analyzer"
    },
    "building_no": {
     "type": "keyword"
    },
    "unit": {
     "type": "text"
    }
   }
  }
 }
}


<h1> INDEXING </h1>

In [3]:
start_time = time.time()
es_connection.load_csv_into_index(index_name=index_name, 
                             path=os.path.join("data", "addresses.csv"))
print("Elapsed time:", time.time() - start_time)

Elapsed time: 37.589699029922485


<h1> SEARCH </h1>

In [6]:
def parse_search_output(s_output):
    hits = s_output["hits"]
    result = []
    if hits["total"]["value"] > 0:
        hits = hits["hits"]
        for hit in hits:
            result.append(hit["_source"])
    return result

In [24]:
def customized_search(address_query='', building_no_query='', unit_query='', address_weight=1, building_no_weight=1, unit_weight=1):
    global es_connection, index_name
    
    es = es_connection.es
    query = {
        "size": 10, 
        "query": {
            "bool": {
                "should":[
                    {"multi_match": {
                        "query": address_query,
                        "type": "bool_prefix",
                        "fields": ["address",
                                   "address._2gram",
                                   "address._3gram"],
                        "boost" : address_weight}},
                    {"match": {
                        "building_no": {
                            "query": building_no_query,
                            "boost" : building_no_weight}}},
                    {"match": {"unit": {
                        "query" : unit_query,
                        "boost" : unit_weight}}}
                ]
            }
        },
        "collapse": {
            "field": "address.keyword"
        }
    }
    
#     query = {
#         "size": 1000, 
#         "query": {
#             "bool": {
#                 "must": [
#                         {
#                             "match_phrase_prefix": {
#                                 "address": {
#                                     "query": address_query
#                                 }
#                             }
#                         }]
#             }
            
#         },
#         "collapse": {
#             "field": "address.keyword"
#         }
#     }
    result = es.search(index=index_name, body=query)
    return parse_search_output(result)

In [25]:
result = customized_search("،دولاب زینتی ")
for r in result:
    print(r)

{'address': 'تهران،پیروزی، خ. زینتی افخم، نرسیده به م. سراسیاب دولاب', 'building_no': '30', 'unit': '1'}
{'address': 'تهران،پیروزی، خ. زینتی افخم، نرسیده به م. سراسیاب دولاب، خراسانی', 'building_no': '33', 'unit': '5'}
{'address': 'تهران،پیروزی، خ. زینتی افخم، نرسیده به م. سراسیاب دولاب، بن. شیرازی', 'building_no': '5', 'unit': '1'}
{'address': 'تهران،پیروزی، خ. زینتی افخم، نرسیده به م. سراسیاب دولاب، خ. خراسانی', 'building_no': '33', 'unit': '5'}
{'address': 'تهران،پیروزی، خ. زینتی افخم، نرسیده به م. سراسیاب دولاب، کوچه خراسانی', 'building_no': '33', 'unit': '5'}
{'address': 'تهران،سراسیاب دولاب خ زینتی افخم کوچه قاسم طوسی کوچه افتخاری', 'building_no': '2', 'unit': '3'}
{'address': 'تهران،پیروزی، خ. زینتی افخم، نبش م. سراسیاب دولاب،کوچه عباسی', 'building_no': '54', 'unit': '2'}
{'address': 'تهران،پیروزی، خ. زینتی افخم، نرسیده به م. سراسیاب دولاب، کوچه اقتداری', 'building_no': '7', 'unit': '1'}
{'address': 'تهران،پیروزی، خ. زینتی افخم، نرسیده به م. سراسیاب دولاب، نبش خ. خراسانی', 'buil

In [23]:
es_connection.es.get(index=index_name, id="jsFHIHsBIrB2tMqiZ09Q")

{'_index': 'address_index',
 '_type': '_doc',
 '_id': 'jsFHIHsBIrB2tMqiZ09Q',
 '_version': 1,
 '_seq_no': 189433,
 '_primary_term': 1,
 'found': True,
 '_source': {'address': 'تهران،پیروزی، خ. زینتی افخم، نرسیده به م. سراسیاب دولاب، خراسانی',
  'building_no': '33',
  'unit': '5'}}